<a id='top'></a>
<h1>PISM paleo data to zarr </h1>

Transformation of data from Antarctic Ice Sheet glacial cycle experiments

(c) 2020 by Torsten Albrecht (torsten.albrecht@pik-potsdam.de) | Potsdam Institute for Climate Impact Research (PIK)

based on https://github.com/ldeo-glaciology/pangeo-bedmachine


*data:* Albrecht, Torsten (2019): PISM parameter ensemble analysis of Antarctic Ice Sheet glacial cycle simulations. PANGAEA, https://doi.pangaea.de/10.1594/PANGAEA.909728

The analysis is based on the scoring scheme with respect to modern and paleo data (https://doi.org/10.5281/zenodo.3585118).

<a id='overview'></a>
<h2>Overview of settings</h2>

**Forcings and initial values** see https://github.com/pism/pism-ais

*Initial ice configuration:* Bedmap2, Fretwell et al., 2013

*Atmosphere:* Racmo v.2.3p2, Wessem et al., 2017

*Ocean:* Schmidtko et al. 2014, Zwally et al., 2012

*Basal Heatflux:* Martos et al., 2017



**Key parameters**
```

ecalv = 1e17 m s
hcalv = 75 m
gammaT = 1.0e-5
overturning_coeff = 0.8e6
ppq = 0.75
essa = 0.6
esia = 2.0
till_effo = 0.04
till_dec = 1 mm/yr
pscale = 7 %/K
visc = 0.5e21 Pa s

```

**Run length**

130,000 years

**Horizontal grid resolution**

16000 m x 16000 m (381 x 381 grid points as in initMIP)

<br>
<div class="alert alert-block alert-warning">**PISM code version**
<br><br>
PISMR (basic evolution run mode stable v1.0-123-gf2e24e8 committed by Torsten Albrecht on 2018-01-22<br>
see branch <a href="https://github.com/talbrecht/pism_pik/commits/pism_pik_1.0">https://github.com/talbrecht/pism_pik/commits/pism_pik_1.0</a>,<br>
see release <a href="https://github.com/talbrecht/pism_pik/releases/tag/v1.0-paleo-ensemble">https://github.com/talbrecht/pism_pik/releases/tag/v1.0-paleo-ensemble</a>,<br>
published in <a href="https://doi.org/10.5281/zenodo.3574033">https://doi.org/10.5281/zenodo.3574033</a><br>
</div>

<a id='settings'></a>
<h2>Notebook settings</h2>

In [1]:
%%javascript
// hide presentation toolbar
$('.nbp-app-bar').toggle()

// disable autoscroll in output cells
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Make the notebook cells take almost all available width
# More info:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

<a id='constants'></a>
<h2>PISM constants and pathes</h2>

In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

import matplotlib.pylab as plt
#from matplotlib import cm, colors, rcParams
import netCDF4 as nc
import numpy as np
#import copy
#import os, glob, sys, copy
import pandas as pd
import xarray as xr
import gcsfs
import json
#import warnings
import cftime
import copy, os
from tqdm.notebook import tqdm   # progress bar

### set constants
seconds_per_year = 365.0*24.0*60.0*60.0 #3.1556926e7


trunk = "/p/tmp/albrecht/paleo_ensemble/"
datapath = trunk+"datapub/model_data/"
ensembleout = datapath+"aggregated_data/"

score_names = ["TOTAL","TOTE","TOTI","TOTDH","TOTVEL","TOTGL","TOTUPL","TROUGH","ELEV","EXT"]

xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
python-bits: 64
OS: Linux
OS-release: 4.4.162-94.72-default
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8
libhdf5: 1.10.4
libnetcdf: 4.7.3

xarray: 0.16.1
pandas: 1.1.4
numpy: 1.19.4
scipy: 1.5.2
netCDF4: 1.5.3
pydap: None
h5netcdf: None
h5py: None
Nio: None
zarr: 2.5.0
cftime: 1.3.0
nc_time_axis: None
PseudoNetCDF: None
rasterio: None
cfgrib: None
iris: None
bottleneck: None
dask: 2.30.0
distributed: 2.30.1
matplotlib: 3.3.3
cartopy: 0.18.0
seaborn: 0.11.0
numbagg: None
pint: None
setuptools: 50.3.0.post20201006
pip: 20.2.4
conda: None
pytest: None
IPython: 7.18.1
sphinx: None


<a id='scores'></a>
<h2>Read the ensemble paramters and scores</h2>

In [4]:
def get_parameter_values_for_ensemble(pfile):

    param_dict={}
    param_ens={}
    param_names=[]
    param_diff={}

    with open(pfile) as f:
      for l,line in enumerate(f.readlines()):

        param_vals={}
        params=line.rstrip('\n').split('"')

        #header with parameter names
        if l==0:
          for pn in params[0].split(' '):
              param_names.append(pn)

        else:
          ensmem=str(params[1].split(' ')[1])
          for pi,pn in enumerate(params[2].split(' ')):
              try:
                param_vals[param_names[pi]]=np.float(pn)
              except:
                "no paramter val"
              if param_names[pi]=='visc':
                param_vals[param_names[pi]]=np.float(pn)*1e21

          if l==1:
            param_diff=param_vals
          else:
            for pn,pval in param_vals.items():
              if pval!=param_diff[pn]:

                if pn not in param_dict.keys():
                  param_dict.setdefault(pn, [])

                if pval not in param_dict[pn]:
                  param_dict[pn].append(pval)

          param_ens[ensmem]=param_vals

    #add parameter vals from first run
    for pn,pvals in param_dict.items():
      param_dict[pn].insert(0,param_diff[pn])

    return (param_ens,param_dict)

##########################################################################################
runfile = ensembleout+"pism1.0_paleo06_6000.csv"
ens_par,par_space = get_parameter_values_for_ensemble(runfile)
#print(ens_par)
#print(par_space)
print(list(par_space.keys()))
parnames = list(par_space.keys())


statfile = ensembleout+"le_all06_16km.txt"

ens_scr={}
with open(statfile) as fp:
    for line in fp:
        scorenum=line.split(",")[0]
        scores=[]
        for i,score in enumerate(line.split(",")):
          if i>0:
            scores.append(np.float(score.rstrip("\n"))) 
        ens_scr[scorenum]=scores
        
best_scores = [ k for k, v in sorted(ens_scr.items(), key=lambda l: l[1], reverse=True)]
print(best_scores[0:3])


par_visc=[]
par_esia=[]
par_prec=[]
par_ppq=[]
scores=[]

for e,p in ens_par.items():
    #print(e,p[parnames[0]])
    par_visc.append(p[parnames[0]])
    par_esia.append(p[parnames[1]])
    par_prec.append(p[parnames[2]])
    par_ppq.append(p[parnames[3]])
    scores.append(ens_scr[str(e)][0])
print(len(par_ppq))


['visc', 'sia_e', 'prec', 'ppq']
['6165', '6245', '6242']
256


<a id='pismfiles'></a>
<h2>Collect all ensemble result files in dict</h2>

In [5]:


ensemble = dict.fromkeys(["name","exp_dir","exp_name","start_id","end_id"])
model_files = ["present","extra","timeseries"]
#model_files = ["present"]
#model_files = ["present","timeseries"]


ensemble['name'] = 'paleo_ensemble'
ensemble['exp_dir'] = datapath
ensemble['exp_name'] = "pism1.0_paleo06"
ensemble['start_id'] = 6000
ensemble['end_id'] = 6255
# best fit/score reference: 6165
ensemble['notes'] = "PISM paleo ensemble data as in https://doi.org/10.1594/PANGAEA.909728"
ensemble['parameter_space'] = par_space

run = dict.fromkeys(["proj_dir","scores","file_path","parameter_values"]) #FIXME: fill notes with parameters and scores 
runs = {}
for id in np.arange(ensemble['start_id'],ensemble['end_id']+1):
    runs[id]=copy.deepcopy(run)
#print(runs)
    

for k, v in tqdm(runs.items(),total=len(runs)):
        #print(k,v)
        runs[k]['proj_dir'] = os.path.join(ensemble['exp_dir'], ensemble['exp_name']+"_"+str(k))
        runs[k]['file_path'] = dict.fromkeys(model_files,"")
        
        present_file = "paleo.nc"
        runs[k]['file_path']['present'] = [os.path.join(runs[k]['proj_dir'], present_file)]
        
        snap_files = ["snapshots_-20000.000.nc","snapshots_-15000.000.nc","snapshots_-10000.000.nc","snapshots_-5000.000.nc",present_file]
        runs[k]['file_path']['snapshots'] = []
        for sf in snap_files:
            runs[k]['file_path']['snapshots'].append(os.path.join(runs[k]['proj_dir'], sf))
        
        mask_file = "extra_paleo_mask.nc"
        runs[k]['file_path']['extra'] = [os.path.join(runs[k]['proj_dir'], mask_file)]
        
        time_file = "timeseries.nc"
        runs[k]['file_path']['timeseries'] = [os.path.join(runs[k]['proj_dir'], time_file)]
        
        runs[k]['parameter_values'] = ens_par[str(k)]
        runs[k]['scores'] = ens_scr[str(k)]
           


<a id='xarrayload'></a>
<h2>Load PISM netCDF files to xarrays </h2>

In [6]:

variables={'snapshots':['thk','mask'],
           'present':['thk','mask','topg','usurf','velsurf_mag','dbdt'],
           'extra':['mask'],
           'lgm':['thk','mask','topg','usurf'],
           'timeseries':['slvol','ice_volume_glacierized','time']}
alldatasets=[]

for mf in tqdm(model_files):
    print(mf)
    if mf != 'timeseries':
        chunks={'x': 381, 'y': 381, 'time': -1}
    else:
        chunks={'time': -1}
    datasets=[]
    for k in sorted(runs.keys()):
        dataset=[]
        if runs[k]['file_path'][mf]:
            for pismf in runs[k]['file_path'][mf]:
              #with xr.set_options(enable_cftimeindex=True):
                    
                with xr.open_dataset(pismf, decode_cf=True,decode_times=False,use_cftime=True,chunks=chunks) as ds: #decode_timedelta=True
                    ds = ds.get(variables[mf])
                    
                    # convert to years
                    ds["time"]=(['time'], ds["time"].time/seconds_per_year)
                    
                    # add ensemble scores
                    ds["score"]=(['id'],  [ens_scr[str(k)][0]])
         
                    # add ensemble paramter space, FIXME: Should be done only once
                    ds.attrs['parameter_space'] = str(par_space)
                    dataset.append(ds)
                    
        concat_time = xr.concat(dataset, dim='time') #,data_vars=list([u'mask',u'thk'])      
        datasets.append(concat_time)
 

    combined = xr.concat(datasets, dim='id')
    
    # workaround to have scores without time dimension
    combined = combined.assign(score=combined["score"])
    combined['score'] = combined['score'].squeeze(drop=True)

    # save the values of id in a data variable before "unstacking", because unstacking will delete id.
    combined = combined.assign(index=combined["id"])
    # make MultiIndex object (this assumes that these do not change over time and that time is the first dimension)
    arrays = list([ par_esia, par_ppq, par_prec, par_visc])
    MI = pd.MultiIndex.from_arrays(arrays, names=('par_esia','par_ppq','par_prec','par_visc'))
    # replace the id coordinate with this MultiIndex Object
    combined.coords['id'] = MI
    # unstack
    combined = combined.unstack(dim='id')
  
    alldatasets.append(combined)

#alldatasets.attrs['parameter_space'] = str(par_space)


present
extra
timeseries



In [7]:
alldatasets

[<xarray.Dataset>
 Dimensions:      (par_esia: 4, par_ppq: 4, par_prec: 4, par_visc: 4, time: 1, x: 381, y: 381)
 Coordinates:
     lat          (y, x) float64 -51.85 -51.94 -52.04 ... -52.04 -51.94 -51.85
     lon          (y, x) float64 -135.0 -135.2 -135.3 -135.5 ... 44.7 44.85 45.0
   * time         (time) float64 50.0
   * x            (x) float64 -3.04e+06 -3.024e+06 ... 3.024e+06 3.04e+06
   * y            (y) float64 -3.04e+06 -3.024e+06 ... 3.024e+06 3.04e+06
   * par_esia     (par_esia) float64 1.0 2.0 4.0 7.0
   * par_ppq      (par_ppq) float64 0.25 0.5 0.75 1.0
   * par_prec     (par_prec) float64 0.02 0.05 0.07 0.1
   * par_visc     (par_visc) float64 1e+20 5e+20 2.5e+21 1e+22
 Data variables:
     thk          (time, y, x, par_esia, par_ppq, par_prec, par_visc) float64 dask.array<chunksize=(1, 381, 381, 1, 4, 4, 4), meta=np.ndarray>
     mask         (time, y, x, par_esia, par_ppq, par_prec, par_visc) int8 dask.array<chunksize=(1, 381, 381, 1, 4, 4, 4), meta=np.ndarray>
 

In [8]:
alldatasets[0].index.isel(par_esia=1,par_visc=1)

<xarray.DataArray 'index' (par_ppq: 4, par_prec: 4)>
array([[  5,  21,  37,  53],
       [ 69,  85, 101, 117],
       [133, 149, 165, 181],
       [197, 213, 229, 245]])
Coordinates:
    par_esia  float64 2.0
  * par_ppq   (par_ppq) float64 0.25 0.5 0.75 1.0
  * par_prec  (par_prec) float64 0.02 0.05 0.07 0.1
    par_visc  float64 5e+20

In [9]:
# This is the reference simulation with score 1, index 165
alldatasets[0].score.isel(par_esia=1,par_visc=1,par_ppq=2,par_prec=2)

<xarray.DataArray 'score' ()>
array(1.)
Coordinates:
    par_esia  float64 2.0
    par_ppq   float64 0.75
    par_prec  float64 0.07
    par_visc  float64 5e+20

<a id='xarraytozarr'></a>
<h2>Export to Zarr Format</h2>

In [11]:
#import subprocess
#
#def du(path):
#    """disk usage in human readable format (e.g. '2,1GB')"""
#    return subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8')

for mi,mf in enumerate(tqdm(model_files)):
    print(mi,mf)
    zarr_folder = trunk+"pangeo/zarr/"+mf
    alldatasets[mi].to_zarr(zarr_folder, consolidated=True, mode='w')
    #print(du(zarr_folder),commands.getoutput('du -sh '+zarr_folder).split()[0])
 

# size of zarr folders:
# 120M	extra
# 682M	present
# 366M	timeseries



0 present
1 extra
2 timeseries



<a id='addtoken'></a>
<h2>Add token from LDEO which enables to write to bucket </h2>


In [12]:
#print(gcsfs.__version__)
with open('../tokens/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
gcs = gcsfs.GCSFileSystem(token=token, access='read_write')

<a id='writezarr'></a>
<h2>Save zarr to bucket </h2>


In [19]:
for mi,mf in enumerate(tqdm(model_files)):
    print(mi,mf)
    #alldatasets[mi] = alldatasets[mi].drop('mapping')   # remove the variable.
    if mf=='timeseries':
        mapper = gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/'+mf+'-2.zarr')
    else:
        mapper = gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/'+mf+'.zarr')
    alldatasets[mi].to_zarr(mapper, consolidated=True, mode='w')

0 present
1 extra
2 timeseries



<a id='checkzarr'></a>
<h2>Reload the zarr and compare with original </h2>


In [20]:
present_reloaded = xr.open_zarr(gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/present.zarr'))  
present_reloaded.identical(alldatasets[0])

True

In [22]:
timeseries_reloaded = xr.open_zarr(gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/timeseries-2.zarr'))  
timeseries_reloaded.identical(alldatasets[2])

True

In [23]:
timeseries_reloaded

<xarray.Dataset>
Dimensions:                 (par_esia: 4, par_ppq: 4, par_prec: 4, par_visc: 4, time: 125050)
Coordinates:
  * par_esia                (par_esia) float64 1.0 2.0 4.0 7.0
  * par_ppq                 (par_ppq) float64 0.25 0.5 0.75 1.0
  * par_prec                (par_prec) float64 0.02 0.05 0.07 0.1
  * par_visc                (par_visc) float64 1e+20 5e+20 2.5e+21 1e+22
  * time                    (time) float64 -1.25e+05 -1.25e+05 ... 49.0 50.0
Data variables:
    ice_volume_glacierized  (time, par_esia, par_ppq, par_prec, par_visc) float64 dask.array<chunksize=(125050, 1, 4, 4, 4), meta=np.ndarray>
    index                   (par_esia, par_ppq, par_prec, par_visc) int64 dask.array<chunksize=(4, 4, 4, 4), meta=np.ndarray>
    score                   (time, par_esia, par_ppq, par_prec, par_visc) float64 dask.array<chunksize=(31263, 1, 2, 2, 2), meta=np.ndarray>
    slvol                   (time, par_esia, par_ppq, par_prec, par_visc) float64 dask.array<chunksize=(125050, 1, 4, 4, 4), meta=np.ndarray>
Attributes:
    Conventions:      CF-1.5
    NCO:              4.6.8
    command:           /p/tmp/albrecht/pism18/pismOut/pism_paleo/pism1.0_pale...
    history:          Fri Dec  6 21:14:20 2019: ncatted -O -a history,global,...
    parameter_space:  {'visc': [1e+20, 5e+20, 2.5e+21, 1e+22], 'sia_e': [1.0,...
    proj4:            +lon_0=0.0 +ellps=WGS84 +datum=WGS84 +lat_ts=-71.0 +pro...
    source:           PISM (stable v1.0-123-gf2e24e88f committed by Torsten A...

<a id='testplot'></a>
<h2>Do some simple plot</h2>

In [25]:
#present_reloaded.mask.isel(par_ppq=2,par_prec=2).plot(x='x',y='y',col='par_esia',row='par_visc')

In [ ]:
#present_reloaded.mask.mean(dim='id').plot()

## Publish notebook
```
cp ~/python/jupyter/paleo/pism_paleo_nc_to_zarr.ipynb ~/www/notebooks/paleo_paper/
see https://nbviewer.jupyter.org/url/www.pik-potsdam.de/~albrecht/notebooks/paleo_paper/pism_paleo_nc_to_zarr.ipynb
```